# Clean up Amazon Personalize resources
This notebook deletes the set of solutions (along with their corresponding
solution versions) and campaigns from Amazon Personalize. It also deletes the event tracker.

Note that the datasets and the dataset group are left in place, as they may be reused. The monthly charges for ongoing storage of imported data, and the data we uploaded to S3, 
are relatively inexpensive.

In [9]:
import json
import boto3
import time

cars_filename         = 'car_items.csv'
users_filename        = 'users.csv'
interactions_filename = 'interactions.csv'

region      = '<region>'
account_num = '<your-account>'
role_arn    = '<your-role>'

dataset_group_name = 'car-dg10'
dataset_group_arn = 'arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, 
                                                                        account_num, 
                                                                        dataset_group_name)

MAX_WAIT_TIME = time.time() + 3*60*60 # 3 hours

In [10]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Define functions to delete campaigns and solutions

In [11]:
def delete_campaign(c):
    try:
        personalize.delete_campaign(campaignArn = c)
    except Exception as e:
        pass
        
def delete_campaigns():
    delete_campaign('arn:aws:personalize:{}:{}:campaign/car-personalized-ranking'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/car-popularity'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/car-sims'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/car-hrnn'.format(region, account_num))
    delete_campaign('arn:aws:personalize:{}:{}:campaign/car-hrnn-metadata'.format(region, account_num))

In [12]:
def delete_solution(s):
    try:
        personalize.delete_solution(solutionArn = s)
    except Exception as e:
        pass
    
def delete_solutions():
    delete_solution('arn:aws:personalize:{}:{}:solution/car-hrnn'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/car-hrnn-metadata'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/car-popularity'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/car-sims'.format(region, account_num))
    delete_solution('arn:aws:personalize:{}:{}:solution/car-personalized-ranking'.format(region, account_num))    

## Do the actual cleanup

In [13]:
delete_campaigns()

In [14]:
time.sleep(30)

In [15]:
delete_solutions()

In [16]:
resp = personalize.list_event_trackers()
trackers = resp['eventTrackers']
for i in range(len(trackers)):
    print('name: {}, arn: {}'.format(trackers[i]['name'], trackers[i]['eventTrackerArn']))
    personalize.delete_event_tracker(eventTrackerArn=trackers[i]['eventTrackerArn'])